In [ ]:
#imports
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [ ]:
# The usual start
load_dotenv(override=True)
openai = OpenAI()
ai_model="gpt-4o-mini"

In [ ]:
# This code will write the user details and questions that cannot be answered by LLM to the files.
user_details_file = "C:/Users/giris/AgenticAIProjects/agents/MyCode/Optima/InterestedUserDetails.txt"
unknown_questions_file = "C:/Users/giris/AgenticAIProjects/agents/MyCode/Optima/UnknownQuestions.txt"

In [ ]:
def write_or_append (filename: str, text: str, encoding: str = "utf-8") -> None:
  mode = "a" if os.path.exists(filename) else "w"
  with open(filename, mode, encoding=encoding) as file:
    file.write(text + "\n") # "\n" will add a new line to the file

In [ ]:
# Tool/Function # 1 to record user details who tried to get in touch
def record_user_details(email, name="Name not provided", notes="not provided"):   
    file_msg=(f"Recording: interest from {name} with email {email} and notes {notes}")
    write_or_append(user_details_file,file_msg)
    return {"recorded": "ok"}

In [ ]:
# Tool/Function #2 to record the question that LLM could not answer
def record_unknown_question(question):
    file_msg=(f"Recording: This question: {question} was asked that I could not answer")
    write_or_append(unknown_questions_file,file_msg)
    return {"recorded": "ok"}

In [ ]:
# Define the response json structure that the LLM will send back for Fuction # 1
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [ ]:
# Define the response json structure that the LLM will send back for Fuction # 2
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
#Now Define the tools / functions that the LLM has options for a response
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [ ]:
#print for debug
#tools
#globals()["record_user_details"]("girish@optimasolutions.us","Girish","Hello - This from python")
#globals()["record_unknown_question"]("This is a hard question")

In [ ]:
# Define how to handle the response back from LLM based on what tool/function the LLM asked us to use
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
        
    return results

In [ ]:
# Now load Optima's Business Description from the pdf
reader = PdfReader("Optima/OptimaBusinessDescription.pdf")
OptimaBusinessDescription = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        OptimaBusinessDescription += text

In [ ]:
# Now Load the Summary provided by Optima in the text file
with open("Optima/OptimaSummary.txt", "r", encoding="utf-8") as f:
    OptimaSummary = f.read()

In [ ]:
#Set Company Name to add to context for Agent
CompanyName = "Optima Business Solutions LLC"

In [ ]:
#Build the System Prompt to set context to Agent to ask the LLM
system_prompt = f"You are acting as a spokeman for {CompanyName}. You are answering questions on {CompanyName}'s website, \
particularly questions related to {CompanyName}'s offerings, background, skills and experience. \
Your responsibility is to represent {CompanyName} for interactions on the website as faithfully as possible. \
You are given a summary of {CompanyName}'s background and Business profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employees who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you \
couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; \
ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{OptimaSummary}\n\n## Business Profile:\n{OptimaBusinessDescription}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {CompanyName}."

In [ ]:
# Now we build the actual chat function.
def chat(user_message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": user_message}]
    # The following while loop will determine if LLM has responded with a tool call or a user response
    ResponseforUser = False
    while not ResponseforUser:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model=ai_model, messages=messages, tools=tools)
     
        # The finish_reason will have the LLM response end status i.e. it the call finished with a tool call or something else. We interpret
        # the something else as a user response
        finish_reason = response.choices[0].finish_reason
           
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            ResponseforUser = True
    return response.choices[0].message.content

In [ ]:
# Now we create the chat interface
gr.ChatInterface(chat, type="messages").launch()